# Agent state occupancy & trajectories

## Init

In [15]:
from pathlib import Path
import numpy as np

In [5]:
save_path = Path("..") / "save"
save_path.absolute()

PosixPath('/home/kir0ul/Projects/RL_Olfaction/cartesian_polar/../save')

In [22]:
sessions_path = save_path / "2025-01-18_16-45-42_EastWest_scratch"
sessions_path.absolute()

PosixPath('/home/kir0ul/Projects/RL_Olfaction/cartesian_polar/../save/2025-01-18_16-45-42_EastWest_scratch')

In [23]:
data_path = sessions_path / "data.npz"
data_path.absolute()

PosixPath('/home/kir0ul/Projects/RL_Olfaction/cartesian_polar/../save/2025-01-18_16-45-42_EastWest_scratch/data.npz')

In [24]:
data_path.exists()

True

In [25]:
data = np.load(data_path)

In [26]:
[item for item in data.keys()]

['rewards',
 'steps',
 'episodes',
 'all_states',
 'all_actions',
 'losses',
 'p',
 'epsilons',
 'weights',
 'biases',
 'weights_val_stats',
 'biases_val_stats',
 'weights_grad_stats',
 'biases_grad_stats',
 'net',
 'env']

## State occupancy

In [28]:
data["all_states"].shape

(11370, 19)

## Trajectories